In [2]:
import pandas as pd
import numpy as np
import math

In [ ]:
import pandas as pd 
import gzip 

def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 

def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
        df[i] = d 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 


## Amazon Movie data

#### Save json.gz as csv

In [ ]:
df_movie_data = getDF('/home/ignacio/Datasets/Amazon/meta_Movies_and_TV.json.gz')
df_movie_data.head()

In [ ]:
len(df_movie_data)

In [ ]:
df_movie_data.to_csv('/home/ignacio/Datasets/Amazon/movie_meta.csv', index=False)

### Clean data: intersect with movies in ml

In [3]:
df_movie_data = pd.read_csv('/home/ignacio/Datasets/Amazon/movie_meta.csv')
df_movie_data.head()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,salesRank,categories,description,title,related,price,imUrl,brand
0,0000143561,{'Movies & TV': 376041},"[['Movies & TV', 'Movies']]","3Pack DVD set - Italian Classics, Parties and ...","Everyday Italian (with Giada de Laurentiis), V...","{'buy_after_viewing': ['B0036FO6SI', 'B000KL8O...",12.99,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN
1,0000589012,{'Movies & TV': 1084845},"[['Movies & TV', 'Movies']]",NaN,Why Don't They Just Quit? DVD Roundtable Discu...,"{'buy_after_viewing': ['B003AC98V2', 'B007VI5A...",15.95,http://ecx.images-amazon.com/images/I/519%2B1k...,NaN
2,0000695009,{'Movies & TV': 1022732},"[['Movies & TV', 'Movies']]",NaN,Understanding Seizures and Epilepsy DVD,NaN,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN
3,000107461X,{'Movies & TV': 954116},"[['Movies & TV', 'Movies']]",NaN,Live in Houston [VHS],NaN,NaN,http://ecx.images-amazon.com/images/I/41WY47gL...,NaN
4,0000143529,{'Movies & TV': 463562},"[['Movies & TV', 'Movies']]",Disc 1: Flour Power (Scones; Shortcakes; South...,My Fair Pastry (Good Eats Vol. 9),"{'buy_after_viewing': ['B0018Y5NVI', 'B003RLLS...",19.99,http://ecx.images-amazon.com/images/I/51QY79CD...,NaN


In [4]:
amazon_titles = df_movie_data['title']
amazon_titles = amazon_titles.dropna()
amazon_titles = amazon_titles.drop_duplicates()
amazon_titles.head()

0    Everyday Italian (with Giada de Laurentiis), V...
1    Why Don't They Just Quit? DVD Roundtable Discu...
2              Understanding Seizures and Epilepsy DVD
3                                Live in Houston [VHS]
4                    My Fair Pastry (Good Eats Vol. 9)
Name: title, dtype: object

In [5]:
amazon_titles = amazon_titles.str.replace(r' ?\[VHS\] ?', '')
amazon_titles = amazon_titles.str.replace(r' ?DVD ?', '')
amazon_titles =amazon_titles.str.lower()
amazon_titles.head()

0    everyday italian (with giada de laurentiis), v...
1    why don't they just quit?roundtable discussion...
2                  understanding seizures and epilepsy
3                                      live in houston
4                    my fair pastry (good eats vol. 9)
Name: title, dtype: object

In [6]:
amazon_titles[amazon_titles.str.contains('^toy story 2')].to_dict()

{31515: 'toy story 2',
 31739: 'toy story 2 (mandarin chinese edition)',
 49289: 'toy story 2 disney read-along'}

In [7]:
df_ml_movies = pd.read_csv('/home/ignacio/Datasets/Graph analysis/ml-valid-movies.csv')
df_ml_movies.head()

,movieId,title,year,genres,director,valid
0,1,Toy Story,1995,adventure|animation|children|comedy|fantasy,"John Lasseter, Lee Unkrich",True
1,7,Sabrina,1995,comedy|romance,"Billy Wilder, Sydney Pollack",True
2,12,Dracula: Dead and Loving It,1995,comedy|horror,Mel Brooks,True
3,16,Casino,1995,crime|drama,"Martin Scorsese, Martin Campbell",True
4,18,Four Rooms,1995,comedy,"Allison Anders, Alexandre Rockwell, Quentin Ta...",True


In [8]:
ml_titles = df_ml_movies['title']
ml_titles = ml_titles.str.lower()
ml_titles = ml_titles.drop_duplicates()
ml_titles.head()

0                      toy story
1                        sabrina
2    dracula: dead and loving it
3                         casino
4                     four rooms
Name: title, dtype: object

In [9]:
len(ml_titles)

518

In [10]:
import re

def exist_in_ml(x, amazon_titles):
    #print(x)
    pattern = re.compile('^' + x + '$')
    result = amazon_titles[amazon_titles.str.contains(pattern)]
    return result.to_dict()

In [11]:
intersect_movies = ml_titles.apply(exist_in_ml, amazon_titles=amazon_titles)
intersect_movies.head()

0    {98328: 'toy story', 24895: 'toy story'}
1        {24601: 'sabrina', 28774: 'sabrina'}
2                                          {}
3                            {2103: 'casino'}
4                       {24842: 'four rooms'}
Name: title, dtype: object

In [12]:
list(intersect_movies[0].keys())

[98328, 24895]

In [13]:
intersect_keys = intersect_movies.apply(lambda x : list(x.keys()) )
intersect_keys.head()

0    [98328, 24895]
1    [24601, 28774]
2                []
3            [2103]
4           [24842]
Name: title, dtype: object

In [14]:
idx = intersect_keys.values
idx_list = [x for index in idx for x in index]

In [15]:
idx_serie = pd.Series(idx_list, name='idx')
idx_serie.head()

0    98328
1    24895
2    24601
3    28774
4     2103
Name: idx, dtype: int64

In [16]:
idx_serie=idx_serie.drop_duplicates()

In [17]:
df_moves_valid = df_movie_data.iloc[idx_serie]
df_moves_valid.head()

,asin,salesRank,categories,description,title,related,price,imUrl,brand
98328,B000GHDPXQ,{'Movies & TV': 985449},"[['Movies & TV', 'Movies']]",NaN,Toy Story,NaN,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN
24895,6304089767,{'Movies & TV': 119681},"[['Movies & TV', 'Movies']]",In the first full-length computer-animated mov...,Toy Story [VHS],"{'bought_together': ['B00003CX7L', 'B00005JKDQ...",3.41,http://ecx.images-amazon.com/images/I/51RC8ZR6...,NaN
24601,6304044836,{'Movies & TV': 353633},"[['Movies & TV', 'Movies']]",Julia Ormond faced one of the great challenges...,Sabrina [VHS],"{'also_bought': ['0767800451', '6304178352']}",5.43,http://ecx.images-amazon.com/images/I/71YBX7Z9...,NaN
28774,6305110859,{'Movies & TV': 422488},"[['Movies & TV', 'Movies']]",Audrey Hepburn is the delightful young Sabrina...,Sabrina,"{'buy_after_viewing': ['B00BTYU3ZS', 'B00AEBB8...",8.00,http://ecx.images-amazon.com/images/I/4146E7JT...,NaN
2103,0783225792,{'Movies & TV': 158336},"[['Movies & TV', 'Movies']]",Director Martin Scorsese reunites with members...,Casino,"{'buy_after_viewing': ['B000P0J09M', 'B000C20V...",2.99,http://ecx.images-amazon.com/images/I/712YQJ01...,NaN


In [18]:
len(df_moves_valid)

394

In [19]:
df_moves_valid.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_ii.csv', index=False)

#### movies intersection

In [20]:
df_movies_valid = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/movie_meta_valid_ii.csv')
df_movies_valid.head()

,asin,salesRank,categories,description,title,related,price,imUrl,brand
0,B000GHDPXQ,{'Movies & TV': 985449},"[['Movies & TV', 'Movies']]",NaN,Toy Story,NaN,NaN,http://g-ecx.images-amazon.com/images/G/01/x-s...,NaN
1,6304089767,{'Movies & TV': 119681},"[['Movies & TV', 'Movies']]",In the first full-length computer-animated mov...,Toy Story [VHS],"{'bought_together': ['B00003CX7L', 'B00005JKDQ...",3.41,http://ecx.images-amazon.com/images/I/51RC8ZR6...,NaN
2,6304044836,{'Movies & TV': 353633},"[['Movies & TV', 'Movies']]",Julia Ormond faced one of the great challenges...,Sabrina [VHS],"{'also_bought': ['0767800451', '6304178352']}",5.43,http://ecx.images-amazon.com/images/I/71YBX7Z9...,NaN
3,6305110859,{'Movies & TV': 422488},"[['Movies & TV', 'Movies']]",Audrey Hepburn is the delightful young Sabrina...,Sabrina,"{'buy_after_viewing': ['B00BTYU3ZS', 'B00AEBB8...",8.00,http://ecx.images-amazon.com/images/I/4146E7JT...,NaN
4,0783225792,{'Movies & TV': 158336},"[['Movies & TV', 'Movies']]",Director Martin Scorsese reunites with members...,Casino,"{'buy_after_viewing': ['B000P0J09M', 'B000C20V...",2.99,http://ecx.images-amazon.com/images/I/712YQJ01...,NaN


In [ ]:
len(df_movies_valid)

#### Movie ratings valid

In [21]:
df_movie_rating = pd.read_csv('/home/ignacio/Datasets/Amazon/ratings_Movies_and_TV.csv', header=None, names=['reviewerID', 'asin', 'rating', 'reviewTime'])
df_movie_rating.head()

,reviewerID,asin,rating,reviewTime
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [22]:
len(df_movie_rating)

4607047

In [23]:
df_movie_rating_valid = df_movie_rating[df_movie_rating['asin'].isin(df_movies_valid['asin'])]
df_movie_rating_valid.head()

,reviewerID,asin,rating,reviewTime
9337,A1DJ7NEM06M0EK,0764003828,5.0,1286236800
9338,AGEIT17HENDIS,0764003828,3.0,1185062400
9339,A1GGOC9PVDXW7Z,0764003828,2.0,1280016000
9340,AEC7JTYYQIGBF,0764003828,4.0,1190851200
9341,A2FY3784R6YDLQ,0764003828,4.0,1399680000


In [24]:
len(df_movie_rating_valid)

67193

In [25]:
df_movie_rating_valid.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movies_valid_ii.csv', index=False)

## Amazon Book data

#### Save json.gz as csv

In [ ]:
df_book_data = getDF(path='/home/ignacio/Datasets/Amazon/meta_Books.json.gz')
df_book_data.head()

In [ ]:
len(df_book_data)

In [ ]:
df_book_data.to_csv('/home/ignacio/Datasets/Amazon/book_meta.csv', index=False)

### Clean data: intersect with goodreads books

In [ ]:
df_book_data = pd.read_csv('/home/ignacio/Datasets/Amazon/book_meta.csv')
df_book_data.head()

In [ ]:
df_book_valid = pd.read_csv('/home/ignacio/Datasets/Graph analysis/bx-valid-book.csv')
df_book_valid.head()

In [ ]:
amazon_book_title = df_book_data['title']
amazon_book_title = amazon_book_title.dropna()
amazon_book_title = amazon_book_title.str.lower()
amazon_book_title = amazon_book_title.str.replace(r'[\]\(\)\\\'\":;]', '')
amazon_book_title = amazon_book_title.str.replace(r'\[', ' ')
amazon_book_title = amazon_book_title.drop_duplicates()
amazon_book_title.head()

In [ ]:
len(amazon_book_title)

In [ ]:
bx_book_title = df_book_valid['Book-Title']
bx_book_title = bx_book_title.dropna()
bx_book_title = bx_book_title.str.lower()
bx_book_title = bx_book_title.str.replace(r'[\(\)\\\'\":;]', '')
bx_book_title = bx_book_title.drop([11911])
bx_book_title = bx_book_title.drop_duplicates()
bx_book_title.head()

In [ ]:
len(bx_book_title)

In [ ]:
frame = 2000
max_frames = math.floor(len(bx_book_title)/frame)
max_frames

In [ ]:
for i in np.arange(5, max_frames):
    s_aux = bx_book_title[ i*frame:(i+1)*frame ].copy()
    intersect_book = s_aux.apply(exist_in_ml, amazon_titles=amazon_book_title)
    intersect_book.to_csv('/home/ignacio/Datasets/Graph analysis/intersect-book'+ str(i) +'.csv', index=False)
    print('Iteration: '+ str(i))

if len(bx_book_title) % frame != 0:
    s_aux = bx_book_title[ max_frames*frame:len(bx_book_title) ].copy()
    intersect_book = s_aux.apply(exist_in_ml, amazon_titles=amazon_book_title)
    intersect_book.to_csv('/home/ignacio/Datasets/Graph analysis/intersect-book'+ str(max_frames) +'.csv', index=False)

In [ ]:
df_book_intersection = pd.read_csv('/home/ignacio/Datasets/Amazon/Book selection/intersect-book0.csv', header=None, names=['titles'])
df_book_intersection.head()

In [ ]:
for i in np.arange(1, 11):
    df_aux = pd.read_csv('/home/ignacio/Datasets/Amazon/Book selection/intersect-book'+str(i)+'.csv', header=None, names=['titles'])
    df_book_intersection = pd.concat([df_book_intersection, df_aux])
len(df_book_intersection)

In [ ]:
book_intersection = df_book_intersection['titles']
book_intersection = book_intersection.str.replace(r'[\{\}]', '')
book_intersection = book_intersection[book_intersection != '']
book_intersection.head()

In [ ]:
len(book_intersection)

In [ ]:
book_intersection.to_csv('/home/ignacio/Datasets/Amazon/Book selection/book-intersect.csv', index=False)

In [ ]:
index_intersection = book_intersection.str.extract(r'(?P<index>\d+):')
index_intersection.head()

In [ ]:
index_intersection = index_intersection.apply(lambda x: int(x))

In [ ]:
df_book_intersection = df_book_data.iloc[index_intersection]
df_book_intersection.head()

In [ ]:
df_book_intersection.to_csv('/home/ignacio/Datasets/Amazon/book-meta-valid.csv', index=False)

#### Book meta valid

In [ ]:
book_meta_valid = pd.read_csv('/home/ignacio/Datasets/Amazon/book-meta-valid.csv')
book_meta_valid.head()

#### Book ratings valid

In [ ]:
book_ratings = pd.read_csv('/home/ignacio/Datasets/Amazon/ratings_Books.csv', header=None, names=['reviewerID', 'asin', 'rating', 'reviewTime'])
book_ratings.head()

In [ ]:
df_books_ratings_valid = book_ratings[book_ratings['asin'].isin(book_meta_valid['asin'])]
df_books_ratings_valid.head()

In [ ]:
len(df_books_ratings_valid)

In [ ]:
df_books_ratings_valid.to_csv('/home/ignacio/Datasets/Amazon/ratings_book_valid.csv', index=False)

### Users in books and movie ratings 

In [26]:
df_movie_ratings = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movies_valid_ii.csv')
df_movie_ratings.head()

,reviewerID,asin,rating,reviewTime
0,A1DJ7NEM06M0EK,0764003828,5.0,1286236800
1,AGEIT17HENDIS,0764003828,3.0,1185062400
2,A1GGOC9PVDXW7Z,0764003828,2.0,1280016000
3,AEC7JTYYQIGBF,0764003828,4.0,1190851200
4,A2FY3784R6YDLQ,0764003828,4.0,1399680000


In [27]:
len(df_movie_ratings)

67193

In [28]:
df_book_ratings = pd.read_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_book_valid.csv')
df_book_ratings.head()

,reviewerID,asin,rating,reviewTime
0,A17KD5LJUQOJHT,0002051850,5.0,999216000
1,A2ILMQSVK5KRTC,0002051850,1.0,1362614400
2,A361FWZ1HA9IPJ,0002051850,4.0,1213833600
3,A2I741VZ6F4RAJ,0002051850,5.0,1363910400
4,AOG084TKLL3X8,0002051850,4.0,1188432000


In [29]:
len(df_book_ratings)

223695

In [30]:
users_movie = df_movie_ratings['reviewerID']
users_movie = users_movie.drop_duplicates()
users_movie.head()

0    A1DJ7NEM06M0EK
1     AGEIT17HENDIS
2    A1GGOC9PVDXW7Z
3     AEC7JTYYQIGBF
4    A2FY3784R6YDLQ
Name: reviewerID, dtype: object

In [31]:
users_book = df_book_ratings['reviewerID']
users_book = users_book.drop_duplicates()
users_book.head()

0    A17KD5LJUQOJHT
1    A2ILMQSVK5KRTC
2    A361FWZ1HA9IPJ
3    A2I741VZ6F4RAJ
4     AOG084TKLL3X8
Name: reviewerID, dtype: object

In [32]:
len(users_book)

165179

In [33]:
users_intersect = users_movie[users_movie.isin(users_book)]
users_intersect.head()

1      AGEIT17HENDIS
2     A1GGOC9PVDXW7Z
5     A34PAZQ73SL163
8     A20IIR0422G3A5
12     AW3VZ5O895LRK
Name: reviewerID, dtype: object

In [34]:
len(users_intersect)

4866

#### Movie ratings common users

In [35]:
df_movie_ratings_intersect = df_movie_ratings[df_movie_ratings['reviewerID'].isin(users_intersect)]
df_movie_ratings_intersect.head()

,reviewerID,asin,rating,reviewTime
1,AGEIT17HENDIS,0764003828,3.0,1185062400
2,A1GGOC9PVDXW7Z,0764003828,2.0,1280016000
5,A34PAZQ73SL163,0764003828,5.0,1110585600
8,A20IIR0422G3A5,0764003828,5.0,1115078400
12,AW3VZ5O895LRK,0764003828,4.0,1379116800


In [36]:
len(df_movie_ratings_intersect)

12471

In [37]:
df_movie_ratings_intersect.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_movie_intersect_ii.csv', index=False)

#### Book ratings common users

In [ ]:
df_book_ratings_intersect = df_book_ratings[df_book_ratings['reviewerID'].isin(users_intersect)]
df_book_ratings_intersect.head()

In [ ]:
len(df_book_ratings_intersect)

In [ ]:
df_book_ratings_intersect.to_csv('/home/ignacio/Datasets/Amazon/Data cleaned/ratings_book_intersect_ii.csv', index=False)